载入库：

In [1]:
from dataclasses import dataclass, asdict
import json

import pandas as pd
import numpy as np

import plotly.express as px
from jupyter_dash import JupyterDash as Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, MATCH, ALL

from tools.obstacle.simulate import ObstacleInfo, obstacle_simulate

In [2]:
app = Dash(__name__)

In [3]:
tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

app.layout = html.Div([
    dcc.Tabs(id="tabs-styled-with-inline", value='obstacle', children=[
        dcc.Tab(label='Obstacle', value='obstacle', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Radar', value='radar', style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Traffic Lane', value='traffic-lane', style=tab_style, selected_style=tab_selected_style)
    ], style=tabs_styles),
    html.Div(id='tabs-content-inline')
])

@app.callback(Output('tabs-content-inline', 'children'),
              Input('tabs-styled-with-inline', 'value'))
def render_content(tab):
    if tab == 'obstacle':
        return html.Div([
            html.H3('障碍物'),
            *[html.P([str(ObstacleInfo(**obstacle_simulate()))]) for _ in range(50)]
        ])
    elif tab == 'radar':
        return html.Div([
            html.H3('雷达')
        ])
    elif tab == 'traffic-lane':
        return html.Div([
            html.H3('车道线')
        ])

In [4]:
app.run_server('external', host='0.0.0.0', port=8077, debug=True)

Dash app running on http://0.0.0.0:8077/


In [ ]:
app.layout = html.Div([
    # The memory store reverts to the default on every page refresh
    dcc.Store(id='memory'),
    # The local store will take the initial data
    # only the first time the page is loaded
    # and keep it until it is cleared.
    dcc.Store(id='local', storage_type='local'),
    # Same as the local store but will lose the data
    # when the browser/tab closes.
    dcc.Store(id='session', storage_type='session'),
    html.Table([
        html.Thead([
            html.Tr(html.Th('Click to store in:', colSpan="3")),
            html.Tr([
                html.Th(html.Button('memory', id='memory-button')),
                html.Th(html.Button('localStorage', id='local-button')),
                html.Th(html.Button('sessionStorage', id='session-button'))
            ]),
            html.Tr([
                html.Th('Memory clicks'),
                html.Th('Local clicks'),
                html.Th('Session clicks')
            ])
        ]),
        html.Tbody([
            html.Tr([
                html.Td(0, id='memory-clicks'),
                html.Td(0, id='local-clicks'),
                html.Td(0, id='session-clicks')
            ])
        ])
    ])
])

# Create two callback for every store.
for store in ('memory', 'local', 'session'):

    # add a click to the appropriate store.
    @app.callback(Output(store, 'data'),
                  Input('{}-button'.format(store), 'n_clicks'),
                  State(store, 'data'))
    def on_click(n_clicks, data):
        if n_clicks is None:
            # prevent the None callbacks is important with the store component.
            # you don't want to update the store for nothing.
            raise PreventUpdate

        # Give a default data dict with 0 clicks if there's no data.
        data = data or {'clicks': 0}

        data['clicks'] = data['clicks'] + 1
        return data

    # output the stored clicks in the table cell.
    @app.callback(Output('{}-clicks'.format(store), 'children'),
                  # Since we use the data prop in an output,
                  # we cannot get the initial data on load with the data prop.
                  # To counter this, you can use the modified_timestamp
                  # as Input and the data as State.
                  # This limitation is due to the initial None callbacks
                  # https://github.com/plotly/dash-renderer/pull/81
                  Input(store, 'modified_timestamp'),
                  State(store, 'data'))
    def on_data(ts, data):
        if ts is None:
            raise PreventUpdate

        data = data or {}

        return data.get('clicks', 0)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

ax = []                    # 定义一个 x 轴的空列表用来接收动态的数据
ay = []                    # 定义一个 y 轴的空列表用来接收动态的数据
plt.ion()                  # 开启一个画图的窗口
for i in range(100):       # 遍历0-99的值
    ax.append(i)           # 添加 i 到 x 轴的数据中
    ay.append(i**2)        # 添加 i 的平方到 y 轴的数据中
    plt.clf()              # 清除之前画的图
    plt.plot(ax, ay)        # 画出当前 ax 列表和 ay 列表中的值的图形
    plt.pause(0.1)         # 暂停一秒
    plt.ioff()             # 关闭画图的窗口
plt.show()

In [ ]:
import toml

In [ ]:
c = toml.load('c.toml')
c

In [ ]:
import threading
import time
from queue import Queue


def job(l, q):
    for i in range(len(l)):
        l[i] = l[i]**2
    q.put(l)  # 多线程调用的函数不能用return返回值


def multithreading():
    q = Queue()
    threads = []
    data = [[1, 2, 3], [3, 4, 5], [4, 4, 4], [5, 5, 5]]
    for i in range(4):
        t = threading.Thread(target=job, args=(data[i], q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    results = []
    for _ in range(4):
        results.append(q.get())
    print(results)


if __name__ == '__main__':
    multithreading()

In [ ]:
import multiprocessing as mp
import threading as td

In [ ]:
def job(a, b):
    print('ccccc')

In [ ]:
if __name__ == '__main__':
    p1 = mp.Process(target=job, args=(1, 2))
    p1.start()
    p1.join()import multiprocessing as mp

def job(a,d):
    print('aaaaa')

if __name__=='__main__':
    p1 = mp.Process(target=job,args=(1,2))
    p1.start()
    p1.join()

In [ ]:
mp.Pipe?